In [1]:
import os
import collections

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem
from rdkit.Chem import Crippen, Draw, rdMolDescriptors

# ランダムシードを設定
from numpy.random import seed
seed(0)

In [2]:
# ファイルパスをここで設定する
# 自分のファイル構成に合わせたファイル設定を
filepath = './'

In [3]:
import Function

In [6]:
# 必要なディレクトリの作成


os.makedirs(filepath + 'ward_result', exist_ok=True)
os.makedirs(filepath + 'ward_result/dendrogram', exist_ok=True)
os.makedirs(filepath + 'ward_result/check_similarity', exist_ok=True)
os.makedirs(filepath + 'to_pdf', exist_ok=True)
os.makedirs(filepath + 'to_pdf/csvfile', exist_ok=True)
os.makedirs(filepath + 'to_pdf/htmlfile', exist_ok=True)
os.makedirs(filepath + 'to_pdf/img', exist_ok=True)
os.makedirs(filepath + 'to_pdf/ligands', exist_ok=True)
os.makedirs(filepath + 'to_pdf/mol', exist_ok=True)
os.makedirs(filepath + 'to_pdf/pdffile', exist_ok=True)
os.makedirs(filepath + 'to_pdf/sdffile', exist_ok=True)
os.makedirs(filepath + 'ward_result/AllAtomNum_AllLigandAtomNum', exist_ok=True)
os.makedirs(filepath + 'ward_result/cluster_in_out_simi', exist_ok=True)
os.makedirs(filepath + 'ward_result/cluster_num', exist_ok=True)
os.makedirs(filepath + 'ward_result/deleted', exist_ok=True)
os.makedirs(filepath + 'ward_result/deleted/deleted_fragment', exist_ok=True)
os.makedirs(filepath + 'ward_result/deleted/deleted_ligand', exist_ok=True)
os.makedirs(filepath + 'ward_result/each_repfrag_freq', exist_ok=True)
os.makedirs(filepath + 'ward_result/frag_and_ligand_cnt', exist_ok=True)
os.makedirs(filepath + 'ward_result/frag_and_ligand_cnt/allfrag', exist_ok=True)
os.makedirs(filepath + 'ward_result/frag_and_ligand_cnt/fragkind', exist_ok=True)
os.makedirs(filepath + 'ward_result/frag_and_ligand_cnt/ligand', exist_ok=True)
os.makedirs(filepath + 'ward_result/new_similarity_result', exist_ok=True)
os.makedirs(filepath + 'ward_result/occupy_hist_img', exist_ok=True)
os.makedirs(filepath + 'ward_result/small_simi', exist_ok=True)
os.makedirs(filepath + 'ward_result/ligands', exist_ok=True)
os.makedirs(filepath + 'simi_scatter', exist_ok=True)
os.makedirs(filepath + 'bioisosteres', exist_ok=True)


In [ ]:
#リガンドのフィルタリング完了
#フラグメントと元のリガンドのsdfファイルをdataframeに変換

f_x = [mol for mol in Chem.SDMolSupplier(filepath + 'drug_bankv1/drug_bank_fragment_filtered.sdf') if mol is not None]
l_x = [mol for mol in Chem.SDMolSupplier(filepath + 'drug_bankv1/drug_bank_structures_filtered.sdf') if mol is not None]
fInfo = [mol for mol in Chem.SDMolSupplier(filepath + 'drug_bankv1/drug_bank_output_filtered.sdf') if mol is not None]


In [ ]:
print("含まれるリガンドの数：", len(l_x))
print("フラグメントの数：", len(f_x))
print("outputのリガンドの数: ", len(fInfo))

In [ ]:
#全てのフラグメントとリガンドのフィルタリング
# #リガンドにフィルターをかける
print("リガンドの数: ", len(l_x))

#fragment info のフラグメントたち
fi = []
for mol in fInfo:
    li = mol.GetProp('fragment_info').split(',')
    for f in li:
        m = Chem.MolFromSmiles(f)
        if m is not None:
            m = Chem.RemoveHs(m)
            s = Chem.MolToSmiles(m)
            m = Chem.MolFromSmiles(f)
            s = Chem.MolToSmiles(m)
            fi.append(s)
cnt = collections.Counter(fi)
print(cnt)
li0 = []
for m in f_x:
    for s in Chem.MolToSmiles(m):
        li0.append(cnt[s])
print(li0)

#fragment info の　smiles を　molに直す
mol_info = [Chem.MolFromSmiles(s) for s in fi]

#総フラグメントのフィルタリング
#なぜか何回かやらないと抜けがあったので何回かやってる
print(len(mol_info))
mol_info = Function.filterings(mol_info)
mol_info = Function.filterings(mol_info)
mol_info = Function.filterings(mol_info)
mol_info = Function.filterings(mol_info)
print("フラグメントの総数", len(mol_info))


In [ ]:
#pdfようなので消さない
# 何でか忘れたけどなんか必要
finfo = []
for mol in fInfo:
    fs = mol.GetProp('fragment_info').split(',')
    li = [Chem.MolFromSmiles(s) for s in fs]
    fs = [Chem.MolToSmiles(m) for m in li]
    for f in fs:
        if Chem.MolFromSmiles(f) == None:
            continue
        finfo.append([mol, Chem.MolFromSmiles(f)])

In [ ]:
#フィルタリング完了

#フラグメント総数
all_fragment = mol_info
print("フラグメント総数：", len(all_fragment))

#フラグメントの種類と出現頻度
cnt = collections.Counter([Chem.MolToSmiles(m) for m in all_fragment])

#フラグメントの種類数
frags = [Chem.MolFromSmiles(s) for s in list(cnt.keys())]
print("フラグメント種類数：", len(frags))


#リガンド総数
ligand = fInfo
print("リガンド総数：", len(ligand))

# 以下基本的に使うのはこの３つのリスト
# all_fragment  = 全てのフラグメント
# frags = all_fragmentの重複をなくしたもの。フラグメントの種類
# ligand = 化合物




In [ ]:
df = Function.genDataFrameSortedByAtomNum(frags)
# df['SMILES'] = [Chem.MolToSmiles(m) for m in df['mol']]
# #並び替えたフラグメントをlistとして抽出する
# f_sorted = df["mol"].to_list()
# #フラグメントの描画　svgファイルに保存
Function.sdf_to_svg(df['mol'], None, None, 'fragment_sorted_v5.svg', filepath)
#リガンドの描画　svgファイルに保存
df_li = Function.genDataFrameSortedByAtomNum(ligand)
Function.sdf_to_svg(df_li['mol'], None, None, 'ligand_sorted_v5.svg', filepath)

In [ ]:
len(frags)
frags_smiles=[Chem.MolToSmiles(m) for m in frags]

In [ ]:
# 変数名の意味の説明
# w・・・ward法の意味　最初はkmeansでもやっていたので区別のためにつけた。
# 20とか50とか・・・クラスタリングの数＝代表フラグメント集合の大きさ
# rep・・・代表フラグメント　のいみ




# クラスタ数 20 50 100 150 200で実験した。卒論で発表したのは50のみ
# 基本50のみ

#代表フラグメントのリストの取得
w50_rep = Function.auto_clustring_new(50, frags, 5, filepath)

# 取得した代表フラグメントのリストを分子量の小順に並べ替える
w50_sorted = Function.genDataFrameSortedByAtomNum(w50_rep)

# 原子数順に並び替える
Function.list_to_svg(Function.genDataFrameSortedByAtomNum(w50_rep)["mol"].to_list(), 'ward_result/w50v5_rep_sorted_new.svg', filepath)

#類似度別に中身確認
Function.looklike_fs_to_svg(Function.make_dis(w50_rep, frags), 'w50v5', filepath)


#sdfファイルとpdfファイルを作成
Function.make_pdf(w50_rep, finfo, 'w50', 5, filepath)
Function.make_sdf(w50_rep, "w50", 5, filepath)

In [ ]:
#これは一回で良い
df_max = Function.make_rep_maxF_similarity(w50_rep, frags)
f_dict = Function.make_50minligand_fragment_dict(df_max, ligand)
allL_f_dict = Function.make_allligand_fragment_dict(df_max, ligand)


In [ ]:
# # #リガンドがどんな代表フラグメントで表現できるのか
#必要！消さない！！
i=0
for key, value in allL_f_dict.items():
    li = []
    li.append(key)
    index = list(value.index)
    rep = [rep_mol for rep_mol in list(value['rep_fragment'])]
    simi = [str(simi) for simi in list(value['similarity'])]
    legendss=[]
    for k in range(len(index)):
        if type(rep[k]) is not float:
            li.append(Chem.MolFromSmiles(rep[k]))
            li.append(Chem.MolFromSmiles(index[k]))
            legendss.append("rep_f: "+ rep[k])
            legendss.append(simi[k])
    legendss.insert(0, 'ligand')
    img = Draw.MolsToGridImage(li, maxMols=1000, molsPerRow=5, useSVG=True, subImgSize=(400, 400), legends=legendss)
    place = filepath + 'ward_result/ligands/' + str(i) + '.svg'
    with open(place, mode='w') as f:
        f.write(img.data)
    i+=1



In [ ]:
#評価　　入力は　「代表フラグメント・代表フラグメントを分子量の昇順にソートしたもの・区別のためのファイル名」

# 類似度閾値
rep_threshold = 0.45
#評価をまとめてやるための関数
Function.all_evaluation(w50_rep, w50_sorted, 'w50v5', frags, ligand, all_fragment, rep_threshold, allL_f_dict, cnt, filepath)

In [ ]:
#どの代表フラグメントとも類似度が閾値以下のフラグメントを数えて割合をグラフにしたい
threshold_list = [0.35, 0.40, 0.45, 0.50, 0.55, 0.60]
kind = [4, 27, 182, 477, 754, 928]
all = [4, 30, 320, 1918, 5860, 7246]
small_df = pd.DataFrame(np.array(threshold_list).T, columns=['threshold'])
small_df['frag_kind'] = np.array(kind).T
small_df['frag_kind_per'] = round(small_df['frag_kind']/len(frags) * 100, 2)
small_df['all_frag'] = np.array(all).T
small_df['all_frag_per'] = round(small_df['all_frag']/len(all_fragment) * 100, 2)

plt.figure(figsize=(5, 7))
ax = small_df.plot(x='threshold', y=['frag_kind_per', 'all_frag_per'], kind='bar', rot=0, alpha=0.5, color=['blue', 'orange'], width=0.80)

# グラフの装飾
plt.xlabel('Threshold', fontsize=14)
plt.ylabel('Proportion of fragment types and all fragments', fontsize=14)
ax.legend(['fragment types', 'all fragment'])
plt.savefig(filepath + 'ward_result/small_simi_fragments_graph.png')

# グラフを表示
plt.close('all')
small_df


In [ ]:
# 1226種類のフラグメントに対して、tanimoto類似度とMQN類似度の分布を確認
t = Function.tanimoto_simi(frags)
sij = Function.MQN_Similarity(frags)
tt=[]
for i in range(0, len(t)):
    for j in range(i+1, len(t)):
        tt.append(round(t[i, j], 3))
ssij=[]
for i in range(0, len(sij)):
    for j in range(i+1, len(sij)):
        ssij.append(round(sij[i, j], 3))

simi_dfdf = pd.DataFrame(columns=['tanimoto', 'MQN'])
simi_dfdf['tanimoto'] = np.array(tt).T
simi_dfdf['MQN'] = np.array(ssij).T
simi_dfdf1 = simi_dfdf[simi_dfdf['tanimoto']!=0]

plt.figure(figsize=(15, 15))
g = sns.jointplot(data=simi_dfdf,
                x='tanimoto',
                y='MQN',
                ratio=2,
                s=5,
                kind="scatter",
                joint_kws={'alpha': 0.2},
                marginal_ticks=True,
                marginal_kws={'bins': 50},
                )
g.ax_joint.set_title("Fragment("+str(len(frags))+ ")Similarity scatter")

# 縦軸と横軸のラベルを追加
g.ax_joint.set_xlabel("tanimoto similarity")
g.ax_joint.set_ylabel('MQN simirarity')
g.ax_joint.set_xticks(np.arange(0, 1.1, 0.1))
g.ax_joint.set_yticks(np.arange(0, 1.1, 0.1))
plt.savefig(filepath + 'simi_scatter/tanimoto_MQN_simi_scatter.png', format='png')
plt.show()


In [ ]:
#クラスタ内外でフラグメント同士の類似度の分布を確認
# クラスタ数50の場合


distance_matrix = 1 - np.array(Function.fragment_similarity(frags))

cluster_similarity_dict = {}
for key, molList in Function.ward_clustering(np.array(frags), distance_matrix, 50, 5, filepath).items():
    in_smiles = [Chem.MolToSmiles(m) for m in molList]
    out_smiles = [s for s in frags_smiles if s not in in_smiles]
    in_mols = [Chem.MolFromSmiles(s) for s in in_smiles]
    out_mols = [Chem.MolFromSmiles(s) for s in out_smiles]
    in_simi = Function.fragment_similarity(in_mols)
    out_simi = Function.fragment_simi_rep_allmol(in_mols, out_mols)
    in_result = []
    out_result = []
    #クラスタ内の類似度を取得
    in_n = len(in_simi)
    for i in range(in_n):
        for j in range(i):
            in_result.append(round(in_simi[i][j], 3))
    for li in out_simi:
        for item in li:
            out_result.append(round(item, 3))
    cluster_similarity_dict[key] = [in_result, out_result]

for key, value in cluster_similarity_dict.items():
    plt.close('all')
    plt.figure(figsize=(10, 5))

    plt.hist(value[0], bins=np.arange(0, 1.05, 0.05), alpha=0.3, color=['blue'], density=True)
    plt.hist(value[1], bins=np.arange(0, 1.05, 0.05), alpha=0.3, color=['red'], density=True)

    # グラフの設定
    plt.xlabel('Similarity')
    plt.ylabel('Molcule pare num')
    plt.title('Distribution of Similarity within/between Clusters(cluster number : ' + str(key) + ')')
    fname = str(key) + 'cluster_in_out_simi.png'
    # 凡例の追加
    plt.legend(['within', 'between'])
    plt.savefig(filepath+'ward_result/cluster_in_out_simi/' + fname)
plt.close('all')



In [ ]:
# bioisosteresなフラグメントの類似度の実験


# bioisosteresなフラグメントを調べて集めてdfにした
# それぞれの類似度を計算してみたけどあんまり上手くいかずに断念
['benzene', 'Fluorobenzene', 'bicyclo1.1.1pentane', 'Suquaramide', 'Guanidine', '1-Cyanoguanidine', 'チオ尿素', '尿素', '酢酸',  'L-Glutamic acid', 'GABA', 'Kainic acid', 'AMPA', 'pyrimidine', '1,2,4-Thiadiazole']

bioisosteres_df = pd.DataFrame(
    data={
        'Name': ['benzene', 'Fluorobenzene', 'bicyclo1.1.1pentane', 'Suquaramide', 'Guanidine', '1-Cyanoguanidine', 'Thiourea', 'Urea', 'acetic_acid',  'L-Glutamic_acid', 'GABA', 'Kainic_acid', 'AMPA', 'pyrimidine', '1,2,4-Thiadiazole'],
        'bioisosteres group':[1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4],
        'CAS_RN':['71-43-2', '462-06-6', '311-75-1', '5231-89-0', '113-00-8', '461-58-5', '62-56-6', '57-13-6', '64-19-7', '56-86-0', '56-12-2', '487-79-6', '83643-88-3', '289-95-2', '288-92-6']
        }
)

bioisosteres_df

#CAS番号をSMILESを含む情報に変換する
import pubchempy as pcp
properties = ['CanonicalSMILES']
infos = []
smileses = []
mols = []
for i in bioisosteres_df["CAS_RN"]:
    info = pcp.get_properties(properties, i, 'name')
    infos.append(info)
#SMILESのみを抜き出す
for i in np.arange(len(infos)):
    try:
        smiles = infos[i][0]["CanonicalSMILES"]
        smileses.append(smiles)
        mols.append(Chem.MolFromSmiles(smiles))
    except:
        smiles = "NaN"  #SMILESが定義されない化合物はNaN埋めする
        smileses.append(smiles)
        mols.append(Chem.MolFromSmiles(smiles))

bioisosteres_df["SMILES"] = smileses
bioisosteres_df["MOLS"] = mols

m = bioisosteres_df["MOLS"]
Draw.MolsToGridImage(m, molsPerRow=5, subImgSize=(200, 200), legends=[str(i) for i in bioisosteres_df["bioisosteres group"]])

m = bioisosteres_df.loc[13:14, 'MOLS']
bioisosteres_simi = Function.fragment_similarity(list(m))
Draw.MolsToGridImage(m, molsPerRow=5, subImgSize=(200, 200), legends=[str(i) for i in bioisosteres_df.loc[13:14, "bioisosteres group"]])


In [ ]:
#代表フラグメントの空間
# 全てのフラグメントの重原子と分子量　logpの分布を見るため
rep = w50_rep
rep_s = [Chem.MolToSmiles(m) for m in rep]
frags_s = [Chem.MolToSmiles(m) for m in frags]
result = [Chem.MolFromSmiles(s) for s in list(filter(lambda x:x not in rep_s, frags_s))]

fname = filepath + 'ward_result/' + 'w50v5'
rep_df = pd.DataFrame(rep, columns=['mol'])

rep_df['heavyAtomnum'] = [m.GetNumAtoms() for m in rep]
rep_df['molWeight'] = [round(rdMolDescriptors._CalcMolWt(m), 2) for m in rep]
rep_df['logP'] = [Crippen.MolLogP(m) for m in rep]
rep_df.to_csv(fname + 'rep_df.csv')

allf_df = pd.DataFrame(result, columns=['mol'])
allf_df['heavyAtomnum'] = [m.GetNumAtoms() for m in result]
allf_df['molWeight'] = [round(rdMolDescriptors._CalcMolWt(m), 2) for m in result]
allf_df['logP'] = [Crippen.MolLogP(m) for m in result]
allf_df.to_csv(fname + 'allf_df.csv')

plt.scatter(allf_df['molWeight'], allf_df['logP'],  color='blue', alpha=0.2, label='all fragment', s=10)
plt.scatter(rep_df['molWeight'], rep_df['logP'], color='red', alpha=0.8, label='representative', s=10)
plt.legend()
plt.ylabel('Log P', fontsize=14)
plt.xlabel('Molecular weight', fontsize=14)
plt.savefig(fname + 'logp_MW.png')
plt.close('all')


plt.scatter(allf_df['molWeight'], allf_df['heavyAtomnum'],  color='blue', alpha=0.2, label='all fragment', s=10)
plt.scatter(rep_df['molWeight'], rep_df['heavyAtomnum'], color='red', alpha=0.8, label='representative', s=10)
plt.legend()
plt.ylabel('Log P', fontsize=14)
plt.xlabel('Molecular weight', fontsize=14)
plt.savefig(fname + 'heavyAtomnum_MW.png')
plt.close('all')




In [ ]:
#どの代表フラグメントとも類似度が低いフラグメントを構成フラグメントとする化合物の数をカウント　　追加実験
# 類似度が低いフラグメントたち
small_simi_sorted = df_max[df_max['similarity'] < 0.45].sort_values(by='similarity', ascending=True)
small_simi = [Chem.MolFromSmiles(s) for s in list(small_simi_sorted.index)]
small_simi_rep = [Chem.MolFromSmiles(s) for s in list(small_simi_sorted['rep_fragment'])]
result = [item for sublist in zip(small_simi_rep, small_simi) for item in sublist]
rep = ['rep' for i in range(len(small_simi))]
legends = [item for sublist in zip(rep, [str(round(s, 3)) for s in small_simi_sorted['similarity']]) for item in sublist]
allL_f_dict

In [ ]:
ligand_num = 0
for v in allL_f_dict.values():
    if v['similarity'].min() < 0.45:
        ligand_num += 1
print(len(ligand))
print("類似度の低いフラグメントで構成されている化合物の数：", ligand_num)
print("類似度の低いフラグメントで構成されている化合物の割合：", ligand_num/len(ligand))
